## Import Dataset

In [1]:
import pandas as pd

data = pd.read_csv('data/winequality-red.csv', sep=';')

print(data.shape[0], 'records')
data.head()

1599 records


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


## Data Preprocessing
* Separate the features and labels
* Prepare data for cross-validation
  * All the data is kept in the training set
* Scale the training data

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
fixed acidity           1599 non-null float64
volatile acidity        1599 non-null float64
citric acid             1599 non-null float64
residual sugar          1599 non-null float64
chlorides               1599 non-null float64
free sulfur dioxide     1599 non-null float64
total sulfur dioxide    1599 non-null float64
density                 1599 non-null float64
pH                      1599 non-null float64
sulphates               1599 non-null float64
alcohol                 1599 non-null float64
quality                 1599 non-null int64
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [3]:
X = data.iloc[:, 0:11].values
print('Features:', X.shape)

y = data.iloc[:, 11].values
print('Labels:', y.shape)

Features: (1599, 11)
Labels: (1599,)


In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X)
y_train = y

## Implement the algorithm

In [5]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=300, random_state=0)
print(classifier)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)


## Implement Cross Validation

In [6]:
from sklearn.model_selection import cross_val_score

accuracies = cross_val_score(cv=5, estimator=classifier, X=X_train, y=y_train)
print('Accuracies:', accuracies)
print('Average:', accuracies.mean()*100,'%')
print('Std Deviation:', accuracies.std()*100,'%')

Accuracies: [0.540625  0.559375  0.628125  0.584375  0.5799373]
Average: 57.84874608150471 %
Std Deviation: 2.9324060281103286 %


## Grid Search for Parameter Selection

In [7]:
from sklearn.model_selection import GridSearchCV

grid_params = {
    'n_estimators': [100, 300, 500, 700, 1000],
    'criterion': ['gini', 'entropy'],
    'bootstrap': [True, False]  
}

grid_search = GridSearchCV(
    estimator=classifier,
    param_grid=grid_params,
    scoring='accuracy',
    cv=5, # number of folds
    n_jobs=-1 # all available computing power
)

grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=300, n_jobs=None,
                                              oob_score=False, random_state=0,
                                   

In [8]:
best_parameters = grid_search.best_params_
print('Best parameters:', best_parameters)

best_result = grid_search.best_score_
print('Best accuracy:', best_result)

Best parameters: {'bootstrap': True, 'criterion': 'gini', 'n_estimators': 300}
Best accuracy: 0.5784874608150471


## Create and save the best model

In [9]:
import pickle

model = RandomForestClassifier(n_estimators=300, random_state=0)
print('Best Model:\n', model)

pickle.dump(model, open('model.sav', 'wb'))

Best Model:
 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)
